In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [11]:
order_1 = pd.read_csv('season_1/training_data/order_data/order_data_2016-01-01', delimiter='\t')

In [17]:
poi_1 = pd.read_csv('season_1/training_data/poi_data/poi_data', delimiter='\t')

CParserError: Error tokenizing data. C error: Expected 139 fields in line 52, saw 142


In [ ]:
poi_1.head()